In [1]:
import pandas as pd
import sqlite3
import concurrent.futures
import time
import re

from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer

In [2]:
# text=f"""Account Director of Integrated Account Department PT Huawei Tech Investment, Mukti Ali, akan diperiksa sebagai terdakwa kasus dugaan korupsi proyek BTS 4G Kominfo. Pemeriksaan akan dilakukan pada Senin, 23 Oktober 2023."Agenda sidang kita berikutnya di hari Senin, tanggal 23 Oktober 2023. Setelah saksi dilanjutkan pemeriksaan terdakwa ya," kata ketua hakim Dennie Arsan Fatrika dalam persidangan di PN Tipikor Jakarta, Jalan Bungur Besar Raya, Jakarta Pusat, Jumat (20/10/2023).Dennie mengatakan Mukti Ali juga akan diperiksa sebagai saksi mahkota terlebih dahulu. Kemudian dilanjutkan dengan pemeriksaan sebagai terdakwa."Demikian ya, jadi untuk persidangan berikutnya ditunda sampai dengan hari Senin, tanggal 23 Oktober 2023 ya dengan agenda untuk pemeriksaan saksi, masing-masing terdakwa sebagai saksi untuk perkara terdakwa lainnya dan dilanjutkan pemeriksaan terdakwa," ujarnya.Komisaris PT Solitech Media Sinergy Irwan Hermawan didakwa melakukan korupsi dalam proyek pengadaan base transceiver station (BTS) 4G dan infrastruktur pendukung paket 1, 2, 3, 4, dan 5 Bakti Kementerian Komunikasi dan Informatika tahun 2020-2022. Irwan didakwa merugikan negara Rp 8 triliun.Irwan diadili bersama Account Director of Integrated Account Department PT Huawei Tech Investment Mukti Ali dan Direktur Utama PT Mora Telematika Indonesia Galumbang Menak Simanjuntak. Mereka didakwa dalam berkas terpisah.Dalam dakwaan yang dibacakan jaksa dalam sidang di Pengadilan Tipikor Jakarta Pusat, Selasa (4/7/2023), Irwan beserta Tenaga Ahli Human Development Universitas Indonesia Tahun 2020 Yohan Suryanto, Direktur Utama Bakti Kominfo Anang Achmad Latif dan Galumbang serta Mukti melakukan pertemuan-pertemuan dengan calon kontraktor dan subkontraktor dalam rangka menentukan pelaksana pekerjaan. Pertemuan itu mengatur persyaratan pemilihan penyedia.Kemudian Irwan menentukan pemenang penyedia, yakni Konsorsium Fiber Home PT Telkominfra dan PT Multi Trans Data (PT MTD) untuk Paket 1, 2, lalu Konsorsium PT Lintas Arta, PT Huawei, dan PT Surya Energy Indotama (SEI) untuk Paket 3, serta Konsorsium PT Infra Struktur Bisnis Sejahtera (IBS) dan PT ZTE Indonesia Paket 4, 5."Anang Achmad Latif memerintahkan Ferandi Mirza untuk membentuk tim bayangan, yang terdiri dari Gandi, Avrinson, Maryulis, Edy untuk memastikan Pokja melaksanakan kriteria yang sudah ditentukan Anang Achmad Latif bersama-sama dengan Terdakwa Irwan Hermawan dan Galumbang Menak Simanjuntak," lanjut jaksa.Simak Video 'Sesal Johnny G Plate Proyek BTS Tak Selesai, Tapi Tak Merasa Salah':[Gambas:Video 20detik]Baca halaman selanjutnya>>Halaman 1 2 Selanjutnya bts kominfo kasus bts kominfo kasus korupsi bts kominfo korupsi bts kominfo hukum"""
# content_cleaned = re.sub(r'ADVERTISEMENT', '', text)
# content_cleaned = re.sub(r'Detik News', '', content_cleaned)
# content_cleaned = re.sub(r'CNN News', '', content_cleaned)
# content_cleaned = re.sub(r'KOMPAS.com', '', content_cleaned)
# content_cleaned = re.sub(r'Kompas News', '', content_cleaned)
# content_cleaned = re.sub(r'Gambas', '', content_cleaned)
# content_cleaned = re.sub(r'20detik', '', content_cleaned)
# content_cleaned = re.sub(r'berikutnya', '', content_cleaned)
# content_cleaned = re.sub(r'halaman', '', content_cleaned)
# content_cleaned = re.sub(r'detikcom', '', content_cleaned)
# content_cleaned = re.sub(r'Halaman', '', content_cleaned)




In [3]:
def stem(word):
    # Daftar aturan awalan dan akhiran yang akan dihapus
    awalan = ["meng", "meny", "men", "mem", "me", "peng", "peny", "pen", "pem", "di", "ter", "ke", "ber", "per"]
    akhiran = ["kan", "an", "i"]

    # Daftar kata-kata pengecualian (singkatan atau istilah khusus)
    pengecualian = ["indonesia", "pemilu","ketum"]

    # Jika kata adalah kata pengecualian, kembalikan kata tanpa stemming
    if word in pengecualian:
        return word


    # Hapus awalan
    for awl in awalan:
        if word.startswith(awl):
            word = word[len(awl):]
            break

    # Hapus akhiran
    for akh in akhiran:
        if word.endswith(akh):
            word = word[:-len(akh)]
            break

    # Exception: ubah 'perekonomian' menjadi 'ekonomi'
    if word == 'perekonomian':
        word = 'ekonomi'
    elif word == 'mengakui':
        word = 'akui'

    return word

# Contoh kalimat yang akan di-stem
kalimat = "Aku mengakui Kesalahan Pemilu Sebagai Ketum Umum"

content_cleaned  = kalimat.lower()

# Tokenisasi kalimat menjadi kata-kata
kata_kata = content_cleaned.split()

# Lakukan stemming pada setiap kata
hasil_stemming = [stem(kata) for kata in kata_kata]

# Gabungkan kata-kata yang telah di-stem kembali menjadi kalimat
kalimat_stem = ' '.join(hasil_stemming)

# Tampilkan hasil stemming
print("Kalimat asli:", kalimat)
print("Hasil stemming:", kalimat_stem)


Kalimat asli: Aku mengakui Kesalahan Pemilu Sebagai Ketum Umum
Hasil stemming: aku aku salah pemilu sebaga ketum umum


In [4]:
import nltk
from nltk.tokenize import word_tokenize

def stem(word):
    # Daftar aturan awalan dan akhiran yang akan dihapus
    awalan = ["meng", "meny", "men", "mem", "me", "peng", "peny", "pen", "pem", "di", "ter", "ke", "ber", "per"]
    akhiran = ["kan", "an", "i"]

    # Daftar kata-kata pengecualian (singkatan atau istilah khusus)
    pengecualian = ["indonesia", "pemilu", "ketum"]

    # Jika kata adalah kata pengecualian, kembalikan kata tanpa stemming
    if word.lower() in pengecualian:
        return word

    # Hapus awalan
    for awl in awalan:
        if word.startswith(awl):
            word = word[len(awl):]
            break

    # Hapus akhiran
    for akh in akhiran:
        if word.endswith(akh):
            word = word[:-len(akh)]
            break

    # Exception: ubah 'mengakui' menjadi 'akui'
    if word == 'mengakui':
        word = 'akui'

    return word

# Contoh kalimat yang akan di-stem
kalimat = "Aku mengakui Kesalahan Pemilu Sebagai Ketum Umum"

# Tokenisasi kalimat menggunakan NLTK
kata_kata = word_tokenize(kalimat)

# Lakukan stemming pada setiap kata
hasil_stemming = [stem(kata) for kata in kata_kata]

# Gabungkan kata-kata yang telah di-stem kembali menjadi kalimat
kalimat_stem = ' '.join(hasil_stemming)

# Tampilkan hasil stemming
print("Kalimat asli:", kalimat)
print("Hasil stemming:", kalimat_stem)


Kalimat asli: Aku mengakui Kesalahan Pemilu Sebagai Ketum Umum
Hasil stemming: Aku aku Kesalah Pemilu Sebaga Ketum Umum


In [7]:
from mpstemmer import MPStemmer

stemmer = MPStemmer()

print(stemmer.stem('mengemudi')) # => kemudi
print(stemmer.stem('belajar')) # => ajar
print(stemmer.stem('ngelepas')) # => lepas
print(stemmer.stem('kebayang')) # => bayang

print(stemmer.stem_kalimat('aku mengakui kesalahan pemilu sebagai ketum umum'))

kemudi
ajar
lepas
bayang
aku aku salah milu sebagai tum umum


In [8]:
#import NDETCStemmer library
from NDETCStemmer import NDETCStemmer

#init stemmer
stemmer=NDETCStemmer()

# stemming process
output=stemmer.stem('boleh saya memerah lembu ini')

print(output)
#boleh saya perah lembu ini

print(stemmer.stem('aku mengakui kesalahan pemilu sebagai ketum umum'))
#bibir merah tangan jadi lengket madu


Model missing, downloading new model....



Downloading...
From: https://drive.google.com/uc?id=1DJ_u_xKSXmgS_CsM0xlB5rIznnDVfz-w
To: /home/krisna/ProjectTA/model_ta/env/lib/python3.11/site-packages/NDETCStemmer/Model/w2vec_wiki_id_case
100%|██████████| 26.7M/26.7M [00:07<00:00, 3.72MB/s]


Downloading...
From (original): https://drive.google.com/uc?id=1DQhPp-D3o0e-x3PfJd2Il3vf7wVgZu4J
From (redirected): https://drive.google.com/uc?id=1DQhPp-D3o0e-x3PfJd2Il3vf7wVgZu4J&confirm=t&uuid=23fbe85f-917a-42a9-85a6-5a1b8b6ef11b
To: /home/krisna/ProjectTA/model_ta/env/lib/python3.11/site-packages/NDETCStemmer/Model/w2vec_wiki_id_case.trainables.syn1neg.npy
100%|██████████| 328M/328M [01:25<00:00, 3.84MB/s] 


Downloading...
From (original): https://drive.google.com/uc?id=1zCn5YINEC82cZ1SH-nB4WXUvuELCzKu-
From (redirected): https://drive.google.com/uc?id=1zCn5YINEC82cZ1SH-nB4WXUvuELCzKu-&confirm=t&uuid=16230b4f-5bdd-4416-8a2a-d4b86d9d6953
To: /home/krisna/ProjectTA/model_ta/env/lib/python3.11/site-packages/NDETCStemmer/Model/w2vec_wiki_id_case.wv.vectors.npy
100%|██████████| 328M/328M [01:23<00:00, 3.92MB/s] 



Download Complete
boleh saya perah lembu ini 
aku aku salah milu bagai tum umum 
